#### Introduction to Statistical Learning, Lab 3.2

# Multiple Linear Regression

In the Python environment the most popular libraries for model fitting (and therefore linear regression) *sklearn* and *statsmodels*. The statsmodels library provides a R-style formula-based interface. We will mostly use this interface because it provides more flexibility and better parameter reporting. This has the additional advantage that it maps quite well onto the examples in the ISLR book.  


  - [statsmodels documentation](https://www.statsmodels.org/stable/)
  - [statsmodels formula interface](https://www.statsmodels.org/stable/example_formulas.html)
  - [the formula mini language](https://patsy.readthedocs.io/en/latest/formulas.html#the-formula-language)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from islpy import datasets
sns.set()
%matplotlib inline

#### Data Set

We use the `Boston` data set to demonstrate multiple linear regression.

In [ ]:
boston = datasets.Boston()
boston.head()

#### Model Specification

The `smf.ols()` function builds a statistical *model* prepared for fitting with *ordinary least squares* (ols). This is the type of fit explained in detail in the lecture.

the syntax to use multiple regressors (variables, predictors, features...) is `y~x1+x2+x3`. As in the simple regression with one predictor, a constant term for the intercept is added automatically.

The formula `medv~lstat+age` means we are using `lstat` and `age` as our predictors and `medv` as our dependent variable:

$$ \mathrm{medv} = \beta_0 + \beta_1 \mathrm{lstat} + \beta_2 \mathrm{age}$$

In [ ]:
model = smf.ols(formula='medv~lstat+age', data=boston)

#### Fitting the Model

We *fit* the model to the data by calling the `fit()` method:

In [ ]:
model_fit = model.fit()

#### Fit Result Summary

We can get a comprehensive summary using the `summary()` method. Now we get the results for all three $\beta$ coefficients.

In [ ]:
model_fit.summary()

#### Specific Summary Tables

We can also select a specific table from the summary. For example the fitted coefficients:

In [ ]:
model_fit.summary().tables[1]

#### Fit Result Parameters

Or we can retrieve only the fitted parameters ($\beta_0$ = *intercept*, $\beta_1$ = *lstat*) as a pandas series using the `params` attribute:

In [ ]:
model_fit.params

#### Confidence Intervals

The 95% confidence intervals for the coefficients can be retrieved via the `conf_int()` method:

In [ ]:
model_fit.conf_int()

#### Visualising the Fit Results

With two predictors we can visualise the data and the fit result in a 3D plot. The `seaborn` library does not provide 3D plotting facilities. There is a good reason for that: it is very hard to make informative 3D charts. Most of the time it is much better to think of a good way to visualise the data in 2D.

That said, we want to give at least one example of a 3D chart. Our approach is similar to the one variable case:

  - First produce a 3D scatter plot.
  - Next get a range of predictor values from the plot's x- and y-axis and compute all predictions on a 2D grid.
  - Then use the `plot_surface()` method to overlay the prediction plane of the fitted model.
  
Like in the one variable case, this approach might seem a bit heavy-handed for a linear model (it plots surface segments between many points on the grid, while only very few are necessary). But again it does have the advantage that it works with *any* model!

In particular this also works for the confidence level surfaces which are *not* planes.

What follows is quite a bit of code; making reasonably looking 3D plots is a bit of work. We include a number of different features in this chart so you have a reference to come back to.

In [ ]:
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
sns.reset_defaults()
%matplotlib inline

fig = plt.figure(figsize=(12, 9))
ax = axes3d.Axes3D(fig)

# 3D scatter plot of the raw data
ax.scatter(boston.lstat, boston.age, boston.medv)

# prepare point grids from the ranges of the scatter plot
xs = np.linspace(*ax.get_xlim(), 100)
ys = np.linspace(*ax.get_ylim(), 100)
xv, yv = np.meshgrid(xs, ys, copy=False)
zv = np.zeros((ys.size, xs.size))
lv = np.zeros((ys.size, xs.size))
uv = np.zeros((ys.size, xs.size))

# compute predictions and CI bounds for the rows in the point grids
for idx, y in enumerate(yv):
    pred = model_fit.get_prediction({'lstat': xs, 'age': y}).summary_frame()
    zv[idx] = pred['mean']
    lv[idx] = pred['mean_ci_lower']
    uv[idx] = pred['mean_ci_upper']

# plot the prediction & CI boundary surfaces
ax.plot_surface(xv, yv, zv, alpha=0.4)
ax.plot_surface(xv, yv, lv, alpha=0.2, color='C1')
ax.plot_surface(xv, yv, uv, alpha=0.2, color='C1')

# add contour plot of the CI width to the bottom of the figure
ax.contourf(xv, yv, uv-lv,
            zdir='z',
            offset=ax.get_zlim()[0],
            levels=30,
            antialiased=True,
            cmap=cm.Oranges)

# set figure title and axes labels
ax.set_title('Linear regression on Boston housing data: medv ~ lstat + age')
ax.set_xlabel('lstat')
ax.set_ylabel('age')
ax.set_zlabel('medv')

# specify viewing angle
ax.view_init(15, -70)

Looks cool, but that's a bad reason to do it. Choosing a good viewing angle and the right surface transparencies can be a bit tricky. You can waste a lot of time (and CPU cycles) on this kind of thing. 

Obviously, once we use more than two predictors, this approach to visualisation won't work anymore. You can play tricks with colour coding and so on. But you can do that in 2D as well and the results will be much more readable!

The bottom line is: __*don't make 3D charts unless you absolutely have to*__.

We use 2D projection plots instead. This idea works for any number of variables.

In [ ]:
sns.set()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))

ax1 = sns.scatterplot(x='lstat', y='medv', data=boston, ax=ax1)
ax2 = sns.scatterplot(x='age', y='medv', data=boston, ax=ax2)

lstat = np.linspace(*ax1.get_xlim(), 100)
age = np.linspace(*ax2.get_xlim(), 100)
pred = model_fit.get_prediction({'lstat': lstat, 'age': age}).summary_frame()
lower = pred['mean_ci_lower']
upper = pred['mean_ci_upper']

ax1.plot(lstat, pred['mean'], color='C1', lw=2)
ax1.fill_between(lstat, lower, upper, alpha=0.3)
ax2.plot(age, pred['mean'], color='C1', lw=2)
ax2.fill_between(age, lower, upper, alpha=0.3)

plt.show()

#### Interlude: Utility Functions & Modules

This is Python, so we can wrap repetitive task in functions. Functions (like any Python object) can be defined in notebook code cells.

In [ ]:
def plot_fitted_model(fitted_model, var_name, data=None,
                      ax=None, points=100, scolor='C0', fcolor='C1', alpha=0.3):
    """Make a scatter plot and overlay fit result."""

    model = fitted_model.model

    if data is None:
        data = pd.DataFrame(model.exog, columns=model.exog_names)
        data[model.endog_names] = model.endog

    xs = pd.DataFrame()
    for name in model.exog_names:
        try:
            xs[name] = np.linspace(data[name].min(), data[name].max(), points)
        except KeyError:
            xs[name] = np.ones(points)  # presumably the intercept
    
    pred = fitted_model.get_prediction(xs).summary_frame()
    x = xs[var_name]
    y = pred['mean']
    cil = pred['mean_ci_lower']
    ciu = pred['mean_ci_upper']

    ax = sns.scatterplot(x=var_name, y=model.endog_names,
                         data=data, ax=ax, color=scolor)
    ax.plot(x, y, color=fcolor, lw=2)
    ax.fill_between(x, cil, ciu, color=fcolor, alpha=alpha)

    return ax

Let's reproduce the two plots using the new utility function.

In [ ]:
sns.set()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))
plot_fitted_model(model_fit, 'lstat', ax=ax1)
plot_fitted_model(model_fit, 'age', ax=ax2)
plt.show()

This function will be useful in the future. We create a module in the `utils/` directory that we can import later at any time.

We have to tell Jupyter/Python to look for modules in the `utils/` directory. This can also be configured centrally (in fact we have done this for you on `student` account on the VM). But we want to show you how to do it in a notebook explicitly.

In [ ]:
import sys
sys.path.insert(0, '../../utils')
import plots
help(plots.plot_fitted_model)

#### Many Variables

Back on topic, we now want to perform a linear regression on *all* the variables in the `Boston` data set. It would be tedious to type in all variable names. Since this is a common problem, it is unfortunate that the formula mini language does not provide a shortcut for this (R does).

Fortunately, it is not too hard to construct the formula using string operations.



In [ ]:
formula = 'medv~' + '+'.join(boston.columns.drop('medv'))
model = smf.ols(formula=formula, data=boston)
model_fit = model.fit()

#### WARNING:

The underlying formula parser has some limitations. Formulas with more than a few hundred terms can not be parsed by the current implementation.

In these situations we can build our model as [documented here](https://patsy.readthedocs.io/en/latest/expert-model-specification.html).

We give an example that replicates the formula-based approach used above.  Essentially, we explicitly build the terms and expressions that the parse of a string formula would provide -- we just took out the middle man. This will work for a large number of features without problems.

In [ ]:
from patsy import ModelDesc, Term, LookupFactor, dmatrices
import statsmodels.api as sm

lhs = [Term([LookupFactor('medv')])]
rhs = [Term([])]  # the intercept
for name in boston.columns.drop('medv'):
    rhs.append(Term([LookupFactor(name)]))
md = ModelDesc(lhs, rhs)
y, xs = dmatrices(md, boston, return_type='dataframe')
model = sm.OLS(y, xs)
model_fit = model.fit()

We can also completely bypass the high-level interface provided by `patsy`.

In [ ]:
y = boston['medv']
xs = boston.drop(columns='medv')
xs['Intercept'] = 1
model = sm.OLS(y, xs)
model_fit = model.fit()

This is fine in this case. If some of our variables should be treated are class variables (categories) things quickly become complicated, though. In these cases we want to use `patsy` or the formula API because they support categorical variables.

In summary:

  - Use the formula API to define models whenever you can. It is concise and expressive.
  - Use `patsy` to explicitly build *design matrices* if you need support for class variables on data sets with many ($\sim\mathcal{O}(1000)$) features.
  - Build the matrices yourself if you have a large number ($\sim\mathcal{O}(1000)$) of simple quantitative features.

In [ ]:
model_fit.summary()

We can also access individual quantities from the fit result. For example, the RSE and $R^2$.

The `statsmodels` documentation provides the [full list](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.RegressionResults.html).

In [ ]:
print(f'RSE: {np.sqrt(model_fit.scale):.4f}')
print(f'R^2: {model_fit.rsquared:.4f}')

#### Variance Inflation Factors

To compute *variance inflation factors* we need to import a function from `statsmodels`.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vifs = {n: variance_inflation_factor(model.exog, i) for i, n in zip(
    range(model.exog.shape[1]), model.exog_names)}
vifs

#### Improving the Model

The $p$-values of `age` and `indus` are rather high. We'd therefore like to remove them from the model and update the fit.

There are several ways to do that. We will simply create a new model with the `age` and `indus` variables removed, as is recommended by the `statsmodels` [documentation](https://www.statsmodels.org/dev/pitfalls.html).

This approach might be problematic in terms of performance (as in CPU time) for large data sets. This is not easy to address and we won't cover it.

In [ ]:
formula = 'medv~' + '+'.join(boston.columns.drop(['medv', 'age', 'indus']))
model = smf.ols(formula=formula, data=boston)
model_fit = model.fit()

In [ ]:
model_fit.summary()

Finally, we make couple of projection plots again; this time using the *imported* plot function.

In [ ]:
sns.set()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.5))
plot_fitted_model(model_fit, 'lstat', ax=ax1)
plot_fitted_model(model_fit, 'dis', ax=ax2)
plt.show()